In [6]:
import requests
from bs4 import BeautifulSoup
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from textblob import TextBlob
# Import API key from config.py
from config import API_KEY

In [7]:
# Function to fetch video comments
def get_video_comments(video_id):
    try:
        youtube = build('youtube', 'v3', developerKey=API_KEY)
        results = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            textFormat='plainText'
        ).execute()
        comments = []
        for item in results['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)
        return comments
    except HttpError as e:
        print(f'Error fetching comments: {str(e)}')
        return []


In [8]:
# Function to perform sentiment analysis
def analyze_sentiment(comments):
    positive = 0
    neutral = 0
    negative = 0
    total_comments = len(comments)
    
    if total_comments == 0:
        return {
            'Positive': 0,
            'Neutral': 0,
            'Negative': 0,
        }
    
    for comment in comments:
        analysis = TextBlob(comment)
        sentiment = analysis.sentiment.polarity
        if sentiment > 0:
            positive += 1
        elif sentiment < 0:
            negative += 1
        else:
            neutral += 1
    
    return {
        'Positive': (positive / total_comments) * 100,
        'Neutral': (neutral / total_comments) * 100,
        'Negative': (negative / total_comments) * 100,
    }


In [9]:
# Main function
def main():
    video_url = input('Enter the YouTube video URL: ')
    video_id = video_url.split('v=')[1]
    comments = get_video_comments(video_id)
    sentiment_analysis = analyze_sentiment(comments)
    print('Sentiment Analysis of Comments:')
    for sentiment, percentage in sentiment_analysis.items():
        print(f'{sentiment}: {percentage:.2f}%')


In [10]:
if __name__ == '__main__':
    main()


Sentiment Analysis of Comments:
Positive: 40.00%
Neutral: 50.00%
Negative: 10.00%
